In [1]:

import ast
import re
from collections import defaultdict
from difflib import SequenceMatcher

import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit

# ========================
# ПАРАМЕТРЫ
# ========================

CSV_PATH = "../datasets/train_raw.csv"      # входной файл
SEP = ";"                       # разделитель
VAL_SIZE = 0.4                 # доля валидации
RANDOM_STATE = 42

# Кластеризация похожих форм
RATIO_THR = 0.7               # порог похожести SequenceMatcher (0..1)
MIN_PREFIX = 4                  # минимальная длина префикс-совпадения
BUCKET_PREFIX_LEN = 3          # сколько символов без пробелов берем в ключ "корзины" для ускорения

# Какие метки считаем валидными сущностями (BIO-суффиксы)
VALID_LABELS = {"BRAND", "TYPE", "VOLUME", "PERCENT"}

# Для каких меток делаем строгую защиту от утечки (рекомендуется оставить только TYPE)
STRICT_LABELS = {"TYPE"}

# Пытаться дополнительно уменьшить утечки по BRAND,
# перенося меньшую сторону перекрывающегося бренд-кластера ТОЛЬКО среди строк без TYPE.
REDUCE_BRAND_LEAK = True


# ========================
# ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# ========================

def parse_ann_cell(cell):
    """annotation -> список (start, end, 'TAG')"""
    try:
        return ast.literal_eval(cell)
    except Exception:
        return []

def strip_bio(tag):
    """'B-TYPE' -> 'TYPE'; 'O' -> 'O'"""
    if not isinstance(tag, str):
        return "O"
    if "-" in tag:
        _, suf = tag.split("-", 1)
        return suf or "O"
    return tag

rus_to_lat_map = str.maketrans({"ё": "е"})
def normalize_token(t: str) -> str:
    return t.strip()

def normalize_entity(text: str) -> str:
    return " ".join(normalize_token(tok) for tok in text.split())

def extract_entities_from_row(text: str, ann_list):
    """Вернёт [(entity_text, label_suf), ...] без 'O', только из VALID_LABELS."""
    ents = []
    for (s, e, tag) in ann_list:
        lab = strip_bio(tag)
        if lab in VALID_LABELS:
            ents.append((text[s:e], lab))
    return ents

class UnionFind:
    def __init__(self, items):
        self.parent = {x: x for x in items}
        self.rank = {x: 0 for x in items}
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra == rb:
            return False
        if self.rank[ra] < self.rank[rb]:
            ra, rb = rb, ra
        self.parent[rb] = ra
        if self.rank[ra] == self.rank[rb]:
            self.rank[ra] += 1
        return True

def bucket_key(s: str) -> str:
    return (s.replace(" ", ""))[:BUCKET_PREFIX_LEN]

def similar(a: str, b: str, ratio_thr=RATIO_THR, min_prefix=MIN_PREFIX) -> bool:
    if a == b:
        return True
    # Префикс (для незаконченного ввода): "стан" vs "станк/станки"
    if a.startswith(b) or b.startswith(a):
        if min(len(a), len(b)) >= min_prefix:
            return True
    # Очень короткие строки — избегаем шумных склеек
    if max(len(a), len(b)) <= 4:
        return False
    r = SequenceMatcher(None, a, b).ratio()
    return r >= ratio_thr

def cluster_entities(entities_set, ratio_thr=RATIO_THR, min_prefix=MIN_PREFIX):
    """Кластеризация похожих нормализованных форм внутри одной метки."""
    ents = sorted(list(entities_set))
    uf = UnionFind(ents)
    buckets = defaultdict(list)
    for e in ents:
        buckets[bucket_key(e)].append(e)
    for key, lst in buckets.items():
        n = len(lst)
        if n <= 1:
            continue
        for i in range(n):
            a = lst[i]
            for j in range(i+1, n):
                b = lst[j]
                if similar(a, b, ratio_thr=ratio_thr, min_prefix=min_prefix):
                    uf.union(a, b)
    groups = defaultdict(list)
    for e in ents:
        groups[uf.find(e)].append(e)
    return groups  # {root: [members,...]}

def build_label_groups(entity_rows, label: str):
    """Вернёт:
    - groups: {root: [members]} для данного label
    - ent_to_cluster: {normalized_entity: cluster_id}
    - per_sample_clusters: список множеств cluster_id для каждого сэмпла
    """
    # собрать множество нормализованных форм для данного label
    uniq = set()
    for ents in entity_rows:
        for e_text, lab in ents:
            if lab == label:
                n = normalize_entity(e_text)
                if n:
                    uniq.add(n)
    groups = cluster_entities(uniq)
    ent_to_cluster = {}
    for root, members in groups.items():
        cid = f"{label}|{root}"
        for m in members:
            ent_to_cluster[m] = cid

    per_sample_clusters = []
    for ents in entity_rows:
        cset = set()
        for e_text, lab in ents:
            if lab == label:
                n = normalize_entity(e_text)
                if n in ent_to_cluster:
                    cset.add(ent_to_cluster[n])
        per_sample_clusters.append(cset)
    return groups, ent_to_cluster, per_sample_clusters

def compute_leakage(per_sample_clusters, train_idx, val_idx):
    """Подсчёт утечки на уровне КЛАСТЕРОВ: сколько cluster_id встречается и в train, и в val."""
    tr, vl = set(), set()
    for i in train_idx:
        tr.update(per_sample_clusters[i])
    for i in val_idx:
        vl.update(per_sample_clusters[i])
    return len(tr & vl), len(tr), len(vl)

# ========================
# ЗАГРУЗКА
# ========================

df = pd.read_csv(CSV_PATH, sep=SEP, encoding="utf-8")
print(f"Loaded: {len(df)} rows; columns: {list(df.columns)}")

# извлечь [(text,label), ...] на строку
entity_rows = []
for i, row in df.iterrows():
    ann = parse_ann_cell(row["annotation"])
    entity_rows.append(extract_entities_from_row(row["sample"], ann))

print("Rows with at least one entity:", sum(1 for r in entity_rows if r))

# ========================
# КЛАСТЕРИЗАЦИЯ ПО ЛЕЙБЛАМ
# ========================

label_groups = {}              # label -> {root: [members]}
label_ent_to_cluster = {}      # label -> {norm_ent: cluster_id}
label_per_sample = {}          # label -> [set(cluster_id), ...]

for lab in VALID_LABELS:
    groups, e2c, per_sample = build_label_groups(entity_rows, lab)
    label_groups[lab] = groups
    label_ent_to_cluster[lab] = e2c
    label_per_sample[lab] = per_sample
    total_entities = sum(len(v) for v in groups.values())
    print(f"[{lab}] clusters: {len(groups)}; unique norm-forms: {total_entities}; "
          f"avg cluster size: {total_entities/len(groups) if groups else 0:.3f}")

# ========================
# ГРУППЫ ДЛЯ STRICT_LABELS (Union по совместной встречаемости)
# ========================

# Построим Union-Find по КЛАСТЕРАМ выбранных меток: если в одном сэмпле встречаются несколько кластеров метки,
# объединяем их в одну компоненту. Так исключаем утечку на уровне кластеров.
all_groups = set()
for lab in STRICT_LABELS:
    all_groups.update({cid for cset in label_per_sample[lab] for cid in cset})
all_groups = sorted(list(all_groups))
uf = UnionFind(all_groups)

for lab in STRICT_LABELS:
    per_sample = label_per_sample[lab]
    for cset in per_sample:
        if len(cset) > 1:
            cset = list(cset)
            base = cset[0]
            for other in cset[1:]:
                uf.union(base, other)

# Назначаем group-id для КАЖДОЙ СТРОКИ:
# - если у строки есть кластеры любой strict-метки — gid = компонент id (find) одного из кластеров
# - иначе — уникальный NO-STRICT|index (чтобы свободно распределялись и помогали балансировать долю)
groups_for_samples = []
strict_present = set()
for idx in range(len(df)):
    cands = set()
    for lab in STRICT_LABELS:
        cands.update(label_per_sample[lab][idx])
    if cands:
        any_cluster = next(iter(cands))
        gid = uf.find(any_cluster)
        groups_for_samples.append(gid)
        strict_present.add(gid)
    else:
        groups_for_samples.append(f"NO-STRICT|{idx}")

print(f"Strict components: {len(strict_present)}; "
      f"samples with strict label(s): {sum(1 for g in groups_for_samples if not str(g).startswith('NO-STRICT'))}")

# ========================
# ГРУППОВОЙ СПЛИТ
# ========================

gss = GroupShuffleSplit(n_splits=1, test_size=VAL_SIZE, random_state=RANDOM_STATE)
idx_all = np.arange(len(df))
train_idx, val_idx = next(gss.split(X=idx_all, groups=np.array(groups_for_samples)))

print(f"Initial split -> train: {len(train_idx)}, val: {len(val_idx)} "
      f"({len(val_idx)/len(df):.3f} val ratio)")

# Проверим утечки по STRICT_LABELS (должно быть 0)
for lab in STRICT_LABELS:
    leak, tr_c, vl_c = compute_leakage(label_per_sample[lab], train_idx, val_idx)
    print(f"[LEAK CHECK] {lab}: leak={leak} (train clusters={tr_c}, val clusters={vl_c})")

# ========================
# МЯГКАЯ МИНИМИЗАЦИЯ УТЕЧЕК ПО BRAND (не ломая TYPE)
# ========================

if REDUCE_BRAND_LEAK and "BRAND" in VALID_LABELS:
    # 1) посчитаем утечки по BRAND
    brand_leak_before, br_tr, br_vl = compute_leakage(label_per_sample["BRAND"], train_idx, val_idx)

    # Соберём, где какой бренд-кластер встречается
    train_set = set(map(int, train_idx))
    val_set = set(map(int, val_idx))

    # Сопоставление "строка -> множество бренд-кластеров"
    brand_per_sample = label_per_sample["BRAND"]
    type_per_sample = label_per_sample["TYPE"] if "TYPE" in VALID_LABELS else [set()]*len(df)

    # Соберём бренды по сторонам
    train_brand_clusters, val_brand_clusters = defaultdict(list), defaultdict(list)
    for i in range(len(df)):
        if not brand_per_sample[i]:
            continue
        key = tuple(sorted(brand_per_sample[i]))  # допускаем несколько брендов в строке
        if i in train_set:
            train_brand_clusters[key].append(i)
        elif i in val_set:
            val_brand_clusters[key].append(i)

    # Найдём пересечения (бренд-кластер(ы) в обеих выборках)
    brand_overlap_keys = set(train_brand_clusters.keys()) & set(val_brand_clusters.keys())

    # План переносов: переносим меньшую сторону ТОЛЬКО если у строк НЕТ TYPE
    train_mask = np.zeros(len(df), dtype=bool)
    train_mask[train_idx] = True
    moved_to_train = moved_to_val = 0

    for key in brand_overlap_keys:
        tr_inds = train_brand_clusters[key]
        vl_inds = val_brand_clusters[key]
        # меньшая сторона
        src_is_train = len(tr_inds) <= len(vl_inds)
        src_inds = tr_inds if src_is_train else vl_inds
        # проверим, что все переносимые индексы действительно без TYPE
        src_inds_no_type = [i for i in src_inds if not type_per_sample[i]]
        if len(src_inds_no_type) != len(src_inds):
            # есть TYPE — не трогаем, чтобы не сломать гарантию
            continue
        # переносим
        if src_is_train:
            train_mask[np.array(src_inds_no_type, dtype=int)] = False
            moved_to_val += len(src_inds_no_type)
        else:
            train_mask[np.array(src_inds_no_type, dtype=int)] = True
            moved_to_train += len(src_inds_no_type)

    train_idx = np.where(train_mask)[0]
    val_idx = np.where(~train_mask)[0]

    brand_leak_after, _, _ = compute_leakage(label_per_sample["BRAND"], train_idx, val_idx)
    print(f"BRAND leak clusters: before={brand_leak_before} -> after={brand_leak_after}; "
          f"moved_to_train={moved_to_train}, moved_to_val={moved_to_val}")

Loaded: 27249 rows; columns: ['sample', 'annotation']
Rows with at least one entity: 26513
[PERCENT] clusters: 19; unique norm-forms: 19; avg cluster size: 1.000
[VOLUME] clusters: 34; unique norm-forms: 36; avg cluster size: 1.059
[BRAND] clusters: 2051; unique norm-forms: 4205; avg cluster size: 2.050
[TYPE] clusters: 4005; unique norm-forms: 15637; avg cluster size: 3.904
Strict components: 3045; samples with strict label(s): 24498
Initial split -> train: 23979, val: 3270 (0.120 val ratio)
[LEAK CHECK] TYPE: leak=0 (train clusters=2801, val clusters=1204)
BRAND leak clusters: before=434 -> after=191; moved_to_train=287, moved_to_val=119


In [2]:
# выбрать доступные индексы
idx_train = train_idx if 'train_idx2' in globals() else train_idx
idx_val   = val_idx   if 'val_idx2'   in globals() else val_idx

# сформировать выборки
train_out = df.iloc[idx_train].copy()
val_out   = df.iloc[idx_val].copy()

# сохранить
train_out.to_csv("../datasets/train_split.csv", sep=";", index=False, encoding="utf-8")
val_out.to_csv("../datasets/val_split.csv",   sep=";", index=False, encoding="utf-8")

print(f"Saved: train={len(train_out)}, val={len(val_out)}")

Saved: train=24147, val=3102


In [ ]:
# -*- coding: utf-8 -*-
import ast
import re
from collections import defaultdict
import numpy as np
import pandas as pd

# ============ ПАРАМЕТРЫ ============
CSV_PATH      = "../datasets/train_raw.csv"   # путь к твоему csv
SEP           = ";"                         # разделитель
VAL_RATIO     = 0.2                         # доля валидации
RANDOM_STATE  = 42

# Параметры похожести (n-граммы + Jaccard)
NGRAM_N       = 3       # длина шингла (2..4)
JACCARD_THR   = 0.69    # порог схожести (0.55..0.70)
MIN_SHARED    = 3       # мин. общих n-грамм, чтобы проверять пару
MIN_LEN       = 2       # игнорировать очень короткие строки
USE_PREFIX    = True    # быстрый префикс-матч для обрезков
MIN_PREFIX    = 3

# Сохранение (можно выключить)
SAVE_SPLIT    = True
OUT_TRAIN     = "../datasets/train_split.csv"
OUT_VAL       = "../datatets/val_split.csv"

# ============ ЗАГРУЗКА ============
df = pd.read_csv(CSV_PATH, sep=SEP, encoding="utf-8")
assert {"sample", "annotation"}.issubset(df.columns), "Нужны колонки: sample, annotation"

# ============ ПАРСИНГ РАЗМЕТКИ ============
def parse_ann(cell):
    """Строка вида "[(0, 4, 'B-TYPE'), ...]" -> список кортежей."""
    try:
        lst = ast.literal_eval(cell)
        if isinstance(lst, list):
            return lst
    except Exception:
        pass
    return []

def strip_bio(tag):
    """B-TYPE -> TYPE, I-BRAND -> BRAND, 'O' -> 'O'."""
    if not isinstance(tag, str):
        return "O"
    if tag == "O":
        return "O"
    if "-" in tag:
        _, suf = tag.split("-", 1)
        return suf
    return tag

def merge_entities(text, ann_list):
    """
    Склеиваем соседние спаны одной метки (B/I) в цельные сущности.
    ann_list: [(start, end, tag), ...] по символам.
    """
    # отсортировать по start
    spans = []
    for s, e, t in ann_list:
        t2 = strip_bio(t)
        spans.append((int(s), int(e), t2))
    spans.sort(key=lambda x: x[0])

    merged = []
    cur_s = cur_e = None
    cur_lab = None

    for s, e, lab in spans:
        if lab == "O":
            # закрыть текущую сущность, если была
            if cur_lab is not None:
                merged.append((cur_s, cur_e, cur_lab))
                cur_s = cur_e = cur_lab = None
            continue

        if cur_lab is None:
            cur_s, cur_e, cur_lab = s, e, lab
        else:
            if lab == cur_lab and (s <= cur_e + 1):  # допускаем пробел
                cur_e = max(cur_e, e)
            else:
                merged.append((cur_s, cur_e, cur_lab))
                cur_s, cur_e, cur_lab = s, e, lab

    if cur_lab is not None:
        merged.append((cur_s, cur_e, cur_lab))
    # вернем (entity_text, label)
    out = []
    for s, e, lab in merged:
        seg = text[s:e]
        out.append((seg, lab))
    return out

# ============ НОРМАЛИЗАЦИЯ ============
rus_to_lat_map = str.maketrans({"ё": "е"})

def normalize_token(t: str) -> str:
    t = t.lower().translate(rus_to_lat_map)
    t = re.sub(r"\s+", " ", t)
    t = re.sub(r"[^0-9a-zа-я% ]+", "", t)  # оставим буквы/цифры/процент/пробел
    return t.strip()

def normalize_text(s: str) -> str:
    return " ".join(normalize_token(tok) for tok in s.split())

# ============ ИЗВЛЕЧЕНИЕ TYPE-СИГНАТУРЫ ДЛЯ КАЖДОЙ СТРОКИ ============
# signature_type[i] — нормализованный текст всех TYPE-сущностей строки (склеенных),
# если нет TYPE — None

signature_type = []
all_type_terms = set()

for i, row in df.iterrows():
    text = str(row["sample"])
    ann  = parse_ann(row["annotation"])
    ents = merge_entities(text, ann)
    type_pieces = []
    for seg, lab in ents:
        if lab == "TYPE":
            type_pieces.append(normalize_text(seg))
    if type_pieces:
        sig = " ".join(type_pieces)            # можно sorted(set(...)), если хочется уникальности
        sig = re.sub(r"\s+", " ", sig).strip()
        signature_type.append(sig if sig else None)
        if sig:
            all_type_terms.add(sig)
    else:
        signature_type.append(None)

# ============ КЛАСТЕРИЗАЦИЯ TYPE ПО N-ГРАММАМ (Jaccard) ============
def ngrams(s: str, n: int) -> set:
    s2 = f"^{s}$"
    return {s2[i:i+n] for i in range(max(0, len(s2) - n + 1))}

def jaccard(a: set, b: set) -> float:
    inter = len(a & b)
    if inter == 0:
        return 0.0
    return inter / len(a | b)

class UF:
    def __init__(self, xs):
        self.p = {x: x for x in xs}
        self.r = {x: 0 for x in xs}
    def f(self, x):
        if self.p[x] != x:
            self.p[x] = self.f(self.p[x])
        return self.p[x]
    def u(self, a, b):
        ra, rb = self.f(a), self.f(b)
        if ra == rb: return False
        if self.r[ra] < self.r[rb]:
            ra, rb = rb, ra
        self.p[rb] = ra
        if self.r[ra] == self.r[rb]:
            self.r[ra] += 1
        return True

# подготовим шинглы и инвертированный индекс
terms = sorted([t for t in all_type_terms if len(t) >= MIN_LEN])
shing = {t: ngrams(t, NGRAM_N) for t in terms}

inv = defaultdict(list)
for t, ss in shing.items():
    for g in ss:
        inv[g].append(t)

uf = UF(terms)

# подберём кандидатов: считаем число общих шинглов, фильтруем по MIN_SHARED, затем Jaccard
for t in terms:
    cand_counts = defaultdict(int)
    ss = shing[t]
    for g in ss:
        for other in inv[g]:
            if other == t:
                continue
            cand_counts[other] += 1

    for other, c in cand_counts.items():
        if c < MIN_SHARED:
            continue

        # префиксный матч для обрезков
        if USE_PREFIX and (t.startswith(other) or other.startswith(t)):
            if min(len(t), len(other)) >= MIN_PREFIX:
                uf.u(t, other)
                continue

        # финальный Jaccard
        if jaccard(ss, shing[other]) >= JACCARD_THR:
            uf.u(t, other)

# соберём кластеры TYPE: term -> cluster_id
type_cluster_id = {}
root_to_terms = defaultdict(list)
for t in terms:
    r = uf.f(t)
    root_to_terms[r].append(t)
for root, members in root_to_terms.items():
    cid = f"TYPE|{root}"
    for m in members:
        type_cluster_id[m] = cid

# ============ ГРУППЫ ДЛЯ СПЛИТА ============
# группа строки = ее TYPE-кластер; если TYPE нет — уникальная группа на строку
groups = []
no_type_count = 0
for i, sig in enumerate(signature_type):
    if sig is None:
        groups.append(f"NO-TYPE|{i}")
        no_type_count += 1
    else:
        groups.append(type_cluster_id.get(sig, f"TYPE-UNSEEN|{sig}"))

print(f"Строк без TYPE: {no_type_count} из {len(df)}")

# ============ СПЛИТ ПО ГРУППАМ ============
idx_all = np.arange(len(df))
groups_arr = np.array(groups)

try:
    from sklearn.model_selection import GroupShuffleSplit
    gss = GroupShuffleSplit(n_splits=1, test_size=VAL_RATIO, random_state=RANDOM_STATE)
    train_idx, val_idx = next(gss.split(idx_all, groups=groups_arr))
except Exception as e:
    # Fallback без sklearn: набираем целевые группы на валидацию
    print("sklearn недоступен, используем резервный сплит. Ошибка:", e)
    rng = np.random.default_rng(RANDOM_STATE)
    uniq_groups = pd.unique(groups_arr)
    rng.shuffle(uniq_groups)
    counts = pd.Series(groups_arr).value_counts()
    target_val = int(len(df) * VAL_RATIO)
    picked, acc = [], 0
    for g in uniq_groups:
        c = int(counts[g])
        if acc + c <= target_val:
            picked.append(g)
            acc += c
        if acc >= target_val:
            break
    val_mask = np.isin(groups_arr, picked)
    val_idx = np.where(val_mask)[0]
    train_idx = np.where(~val_mask)[0]

print(f"Train: {len(train_idx)}  Val: {len(val_idx)}  (val ~ {len(val_idx)/len(df):.3f})")

# ============ ПРОВЕРКА ОТСУТСТВИЯ УТЕЧКИ ПО TYPE ============
train_types = set()
val_types   = set()
for i in train_idx:
    if signature_type[i] is not None:
        train_types.add(type_cluster_id.get(signature_type[i], signature_type[i]))
for i in val_idx:
    if signature_type[i] is not None:
        val_types.add(type_cluster_id.get(signature_type[i], signature_type[i]))

leak = train_types & val_types
print(f"Пересечение TYPE-кластеров между train/val: {len(leak)}")  # должно быть 0

# ============ СОХРАНЕНИЕ ============
if SAVE_SPLIT:
    df.iloc[train_idx].to_csv(OUT_TRAIN, sep=SEP, index=False, encoding="utf-8")
    df.iloc[val_idx].to_csv(OUT_VAL,   sep=SEP, index=False, encoding="utf-8")
    print(f"Сохранено: {OUT_TRAIN} ({len(train_idx)}), {OUT_VAL} ({len(val_idx)})")


Строк без TYPE: 2751 из 27249
Train: 21959  Val: 5290  (val ~ 0.194)
Пересечение TYPE-кластеров между train/val: 0
Сохранено: train_split.csv (21959), val_split.csv (5290)
